In [ ]:
#library(Seurat)
#library(ggplot2)
#library(patchwork)
library(dplyr)
#library(hdf5r)
#library(png)
#library(ggalluvial)
#library(tidyverse)
#library(clustree)
##library(RCTD)
#library(spacexr)
#library(Matrix)
#library(SPOTlight)
#library(future)
#library(grid)
#library(RColorBrewer)
#library(pheatmap)
#library(circlize)
#library(igraph)
##library(ComplexHeatmap)
library(stringr)
##library(ComplexHeatmap)
#library(cowplot)
library(clusterProfiler)
options(connectionObserver = NULL)
library(biomaRt)
##library(scClustViz)
#library(clusterProfiler)
#options(connectionObserver=NULL)
#library(org.Mm.eg.db)
#library(ggplot2)
#library(scales)

#library(spacexr)
library(Matrix)
library(doParallel)
library(ggplot2)
library(ComplexHeatmap)

In [ ]:
wd<-"/home/jovyan/zxli_SCI/result/RCTD/testC5/WT_all/fcreg1_220503/sham_Astr_DEG/"
if(!dir.exists(wd))
    dir.create(wd)
setwd(wd)
getwd()

In [142]:
get_marker_data <- function(cell_type_names, cell_type_means, gene_list) {
  marker_means = cell_type_means[gene_list,]
  marker_norm = marker_means / rowSums(marker_means)
  marker_data = as.data.frame(cell_type_names[max.col(marker_means)])
  marker_data$max_epr <- apply(cell_type_means[gene_list,],1,max)
  colnames(marker_data) = c("cell_type",'max_epr')
  rownames(marker_data) = gene_list
  marker_data$log_fc <- 0
  epsilon <- 1e-9
  for(cell_type in unique(marker_data$cell_type)) {
    cur_genes <- gene_list[marker_data$cell_type == cell_type]
    other_mean = rowMeans(cell_type_means[cur_genes,cell_type_names != cell_type])
    marker_data$log_fc[marker_data$cell_type == cell_type] <- log(epsilon + cell_type_means[cur_genes,cell_type]) - log(epsilon + other_mean)
  }
  return(marker_data)
}

In [169]:
datadir<-c("/home/jovyan/zxli_SCI/result/RCTD/testC5/WT_all/fcreg1_220503/allreps/")

In [170]:
#reference<-readRDS(paste0(datadir,"Reference.renamed.merged16celltypes.220325.rds"))
#reference<-readRDS("/home/jovyan/zxli_SCI/result/RCTD/ref_10x_astr/L3_taxon/Astrocyte_integrate_split2A/2reference.3astr.labeltransfer.L3.astrocyte_RCTD.reference.seed1234.down1000.without7dpi.rds")
reference<-readRDS(paste0(datadir,"Reference.renamed.merged13celltypes.220507.rds"))

In [ ]:
#decon_f<-"../WT.fcreg1_subweights.decon_conf.nor.merged16celltypes.csv"
#meta<-read.csv("../32celltypes.dec_conf.nor.meta.csv")
meta<-read.csv("../16celltypes.dec_conf.nor.meta.csv")
rownames(meta)<-meta[,1]
meta<-meta[,-1]
head(meta)
colnames(meta)

In [172]:
#celltypes<-c('A.Endothelial','Astro.Svep1','Astrocyte.Gfap','Astrocyte.Slc7a10',
#             'Astroependymal','Border.Associated.Mac','C.Endothelial',
#             'Chemotaxis.Inducing.Mac','Dendritic','Div.OPC','Dividing.Microglia',
#             'Dividing.Myeloid','Ependymal.A','Ependymal.B','Fibroblast',
#             'Homeostatic.Microglia','Inflammatory.Mac','Inflammatory.Microglia',
#             'Interferon.Myeloid','Migrating.Microglia','Monocyte','Neuron',
#             'Neutrophil','Oligodendrocyte','OPC.A','OPC.B','Pericyte','Pre.Oligo',
#             'Tip.Cell','U.Vascular','V.Endothelial','VSMC')

celltypes<-c('Astro.Svep1','Astrocyte.Gfap','Astrocyte.Slc7a10',
             'Dendritic','Fibroblast','Homeostatic.Microglia',
             'Monocyte','Neuron','Neutrophil','Oligodendrocyte',
             'Vascular','OPC','Macrophage','R.Microglia',
             'Myeloid','Ependymal')

In [173]:
weights_df<-meta[,celltypes]

### check astrocyte subtypes spatial distribution specificity

In [ ]:
wm_cells<-rownames(subset(meta,domain=="WM"&orig.ident=="WT_sham_H_R2"))
length(wm_cells)
gm_cells<-rownames(subset(meta,domain%in%c("MG","DH","VH")&orig.ident=="WT_sham_H_R2"))
length(gm_cells)
mg_cells<-rownames(subset(meta,domain%in%c("MG")&orig.ident=="WT_sham_H_R2"))
length(mg_cells)
wdv_cells<-rownames(subset(meta,domain%in%c("WM","DH","VH")&orig.ident=="WT_sham_H_R2"))
length(wdv_cells)

In [ ]:
gfap_cells<-rownames(subset(meta,Astrocyte.Gfap>0&orig.ident=="WT_sham_H_R2"))
length(gfap_cells)

slc_cells<-rownames(subset(meta,Astrocyte.Slc7a10>0&orig.ident=="WT_sham_H_R2"))
length(slc_cells)

In [ ]:
wm_gfap_cells<-intersect(wm_cells,gfap_cells)
length(wm_gfap_cells)
#length(intersect(wdv_cells,gfap_cells))
#length(intersect(gm_cells,gfap_cells))
#length(intersect(mg_cells,gfap_cells))
#length(intersect(mg_cells,slc_cells))
gm_slc_cells<-intersect(gm_cells,slc_cells)
length(gm_slc_cells)

In [ ]:
sub_cells<-unique(c(wm_gfap_cells,gm_slc_cells))
length(sub_cells)

### merge into one cell type

In [ ]:
weights_ad<-weights_df
weights_ad$Astrocyte<-rowSums(weights_ad[,c("Astro.Svep1","Astrocyte.Gfap","Astrocyte.Slc7a10")])
weights_ad<-weights_ad[,-which(colnames(weights_ad)%in%c("Astro.Svep1","Astrocyte.Gfap","Astrocyte.Slc7a10"))]
head(weights_ad)
colnames(weights_ad)

In [256]:
for(i in unique(meta$sample)){
    cells<-meta[meta$sample==i,"X"]
    temp_wt<-weights_ad[cells,]
    write.csv(temp_wt,paste0(datadir,i,".conf_nor.13cp.weights.df.csv"))
}

### load sham_H_1mm 4 replicates

In [19]:
puck1<-readRDS(paste0(datadir,"WT_sham_H_R2_1.puck.rds"#'WT_sham_H_R2_1mm_1.puck.rds'
                     ))

puck2<-readRDS(paste0(datadir,"WT_sham_H_R2_2.puck.rds"#'WT_sham_H_R2_1mm_2.puck.rds'
                     ))

puck3<-readRDS(paste0(datadir,"WT_sham_H_R2_3.puck.rds"#'WT_sham_H_R2_1mm_3.puck.rds'
                     ))

puck4<-readRDS(paste0(datadir,"WT_sham_H_R2_4.puck.rds"#'WT_sham_H_R2_1mm_4.puck.rds'
                     ))


In [ ]:
spatial.replicates <- list(puck1, puck2, puck3, puck4)
replicate_names <- c('1','2','3','4')
group_ids <- c(1,1,1,1)
myRCTD.reps <- create.RCTD.replicates(spatial.replicates,
                                      reference, 
                                      replicate_names, 
                                      group_ids = group_ids,
                                      UMI_min=0,
                                      UMI_max=2000000,
                                      CELL_MIN_INSTANCE=10,
                                      keep_reference = T)

#### get 13cp sc reference deconvolution markers

In [ ]:
sc_df<-get_marker_data(myRCTD.reps@RCTD.reps[[1]]@cell_type_info$info[[2]],myRCTD.reps@RCTD.reps[[1]]@cell_type_info$info[[1]],myRCTD.reps@RCTD.reps[[1]]@internal_vars$gene_list_bulk)
sc_df$gene<-rownames(sc_df)
head(sc_df)

In [ ]:
sc_exp<-myRCTD.reps@RCTD.reps[[1]]@cell_type_info$info[[1]]
#sc_exp$gene<-rownames(sc_exp)
head(sc_exp)
sc_exp_norm<-sc_exp[sc_df$gene,]
dim(sc_exp_norm)
head(sc_exp_norm)
sc_exp_norm<-sc_exp_norm / rowSums(sc_exp_norm)
sc_exp_norm$gene<-rownames(sc_exp_norm)
head(sc_exp_norm)

In [ ]:
table(sc_df$cell_type)
sc_df<-cbind(sc_df,sc_exp_norm[sc_df$gene,])
write.csv(sc_df,paste0(datadir,".Astr_merged.total13celltypes.normalized.df.csv"))

In [ ]:
### visualize Astrocyte markers in deconvolution reference matrix
p_df<-sc_exp_norm[sc_df$gene[sc_df$cell_type=="Astrocyte"],]
#p_df<-melt(p_df)
#colnames(p_df)<-c("gene","celltype","express")
#p_df$gene<-factor(p_df$gene,levels = mdf$gene)
head(p_df)

In [ ]:
options(repr.plot.width=5,repr.plot.height=14)
pdf("Astrocyte.markers.express.in.10Xsc.celltype.marker.heatmap.pdf",width = 5,height = 14)
Heatmap(p_df[,-which(colnames(p_df)=="gene")],show_row_names = F,
        cluster_rows = F ,col=c("#05445E",#"#54BAB9",
                                                                      #"white",
                                                                      "#F6E6E8",
                                                                      #"#E9DAC1",
                                                                      "#E56997",#"#F51720",
                                                                      "#B91646"))
dev.off()

In [ ]:
cells1<-rownames(puck1@coords)
length(cells1)
cells2<-rownames(puck2@coords)
length(cells2)
cells3<-rownames(puck3@coords)
length(cells3)
cells4<-rownames(puck4@coords)
length(cells4)

In [ ]:
weight_df1<-weights_ad[cells1,]
dim(weight_df1)

weight_df2<-weights_ad[cells2,]
dim(weight_df2)

weight_df3<-weights_ad[cells3,]
dim(weight_df3)

weight_df4<-weights_ad[cells4,]
dim(weight_df4)

In [199]:
myRCTD.reps@RCTD.reps[[1]]<-import_weights(myRCTD.reps@RCTD.reps[[1]],weight_df1)
myRCTD.reps@RCTD.reps[[2]]<-import_weights(myRCTD.reps@RCTD.reps[[2]],weight_df2)
myRCTD.reps@RCTD.reps[[3]]<-import_weights(myRCTD.reps@RCTD.reps[[3]],weight_df3)
myRCTD.reps@RCTD.reps[[4]]<-import_weights(myRCTD.reps@RCTD.reps[[4]],weight_df4)

In [200]:
saveRDS(myRCTD.reps,"13cp.sham_H.myRCTD.reps.rds")

### set variables that deviate two astrocyte

In [ ]:
exp.var1<-as.integer(cells1%in%gfap_cells)
#head(exp.var1)
names(exp.var1)<-cells1
head(exp.var1)

exp.var2<-as.integer(cells2%in%gfap_cells)
#head(exp.var1)
names(exp.var2)<-cells2
head(exp.var2)

exp.var3<-as.integer(cells3%in%gfap_cells)
#head(exp.var1)
names(exp.var3)<-cells3
head(exp.var3)

exp.var4<-as.integer(cells4%in%gfap_cells)
#head(exp.var1)
names(exp.var4)<-cells4
head(exp.var4)

In [203]:
exvar_list <- list(exp.var1, exp.var2, exp.var3, exp.var4)

In [28]:
saveRDS(exvar_list,"sham_H.Astr.subtype.exvar.list.rds")

In [ ]:
aggregate_cell_types(myRCTD.reps@RCTD.reps[[1]],barcodes = rownames(myRCTD.reps@RCTD.reps[[1]]@spatialRNA@coords),doublet_mode = F)
aggregate_cell_types(myRCTD.reps@RCTD.reps[[2]],barcodes = rownames(myRCTD.reps@RCTD.reps[[2]]@spatialRNA@coords),doublet_mode = F)
aggregate_cell_types(myRCTD.reps@RCTD.reps[[3]],barcodes = rownames(myRCTD.reps@RCTD.reps[[3]]@spatialRNA@coords),doublet_mode = F)
aggregate_cell_types(myRCTD.reps@RCTD.reps[[4]],barcodes = rownames(myRCTD.reps@RCTD.reps[[4]]@spatialRNA@coords),doublet_mode = F)

In [205]:
cell_types<-c('Astrocyte','Neuron','Oligodendrocyte',"Homeostatic.Microglia","Vascular")

In [ ]:
myRCTD.reps <- run.CSIDE.replicates(myRCTD.reps, exvar_list,population_de = T, 
                                    cell_type_threshold = 1,
                                    cell_types,
                                    doublet_mode = F,#gene_threshold=0.01,
                                    #fdr=0.05,
                                    weight_threshold = 0.9
                                    #,normalize_expr=T
                                   )

In [215]:
saveRDS(myRCTD.reps,"13cp.sham_H_astrocyte.gfap.vs.slc.4reps.cpt1.wt09.CSIDE.rds")

In [3]:
myRCTD.reps<-readRDS("sham_H_astrocyte.gfap.vs.slc.4reps.fdr005.wt01.CSIDE.rds")

In [4]:
de_result<-myRCTD.reps@population_sig_gene_df

In [5]:
test<-de_result$Astrocyte

In [ ]:
test<-arrange(test,log_fc_est)
head(test)
tail(test)

In [222]:
write.csv(test,"13cp.sham_H_astr.gfap.vs.slc7a10_4reps.CSIDE.cpt1.wt09.sig_gene_df.csv")

In [ ]:
test<-read.csv("13cp.sham_H_astr.gfap.vs.slc7a10_4reps.CSIDE.cpt1.wt09.sig_gene_df.csv")
rownames(test)<-test[,1]
test<-test[,-1]
head(test)

In [ ]:
backid<-read.table("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/WT.GOterm.SCT.backgeneids.txt")
backid<-backid$x
head(backid)

In [9]:
sf1<-"goterm_result/"
if(!dir.exists(sf1))
    dir.create(sf1)

In [ ]:
pvalueCutoff=0.05
qvalueCutoff=0.05

#df<-data.frame()
for(i in names(de_result)){
    temp<-de_result[[i]]
    temp<-subset(temp,q_val<0.05&p<0.05)
    if(nrow(temp)>0){
        temp$gene<-rownames(temp)
        temp$group<-NA
        temp[temp$log_fc_est<0,"group"]<-paste0(i,"_sham.MG")
        temp[temp$log_fc_est>0,"group"]<-paste0(i,"_sham.WM")
        if(nrow(temp)>30){
            grlabs<-split(temp$gene,temp$group)
            gcSample<-lapply(grlabs,function(gr) as.numeric(bitr(gr,fromType="SYMBOL",toType = "ENTREZID",OrgDb="org.Mm.eg.db")$ENTREZID))
        
            xx.mus.go<-compareCluster(gcSample,OrgDb="org.Mm.eg.db",fun="enrichGO",
                                      pvalueCutoff=pvalueCutoff,qvalueCutoff=qvalueCutoff,
                                 ont="BP",readable=T,universe=backid)
            if(nrow(xx.mus.go@compareClusterResult)>0){
                saveRDS(xx.mus.go,paste0(sf1,i,"_H.4reps.13cp.sham.WM.vs.MG.cpt1.wt09.deg.pq005.go.rds"))
                #temp2<-xx.mus.go@compareClusterResult
                #temp2$Cluster<-paste0(i,"_",temp2$Cluster)
                #df<-rbind(df,temp2) 
        }
               
    }  
    }
    
}

In [ ]:
go<-readRDS(paste0(sf1,"Astrocyte_H.4reps.13cp.sham.WM.vs.MG.cpt1.wt09.deg.pq005.go.rds"))
temp<-go@compareClusterResult
dim(temp)

In [ ]:
table(temp$Cluster)
range(temp$p.adjust)
table(temp$p.adjust<0.05)
temp<-temp[temp$p.adjust<0.05&temp$Count>=5,]
table(temp$Cluster)

In [ ]:
df1<-subset(temp,Cluster=="Astrocyte_sham.WM")
dim(df1)
df2<-subset(temp,Cluster=="Astrocyte_sham.MG")
dim(df2)

In [ ]:
common<-intersect(df1$Description,df2$Description)
common

In [ ]:
df_ad<-rbind(df1,df2#,df3
            )
df_ad$Cluster<-factor(df_ad$Cluster,levels = c("Astrocyte_sham.WM","Astrocyte_sham.MG"))
df_ad<-df_ad[order(df_ad$Cluster),]
head(df_ad)

In [19]:
top10<-df_ad %>%group_by(.,Cluster) %>% top_n(.,-10,p.adjust) %>% top_n(.,10,Count)
top10<-arrange(top10,Cluster)
top10$Description<-factor(top10$Description,levels=rev(unique(top10$Description))) #order Description by cluster 

In [ ]:
### barplot
options(repr.plot.width=8,repr.plot.height=8)
ggplot(top10,aes(x=Description,y=-log10(p.adjust),
              fill=Cluster))+
    geom_bar(stat="identity",position = "dodge")+
    coord_flip()+
    scale_fill_manual(values = rev(c("#DE8BE6","#911eb4")))+
    theme_bw()+
    #geom_text(aes(label=Count),hjust=1,size=3.5,color="white")+
    theme(axis.text.y.left = element_text(size = 13),panel.background = element_blank(),
          panel.grid.major = element_blank(),panel.grid.minor=element_blank())
ggsave("13cp.Astrocyte_sham_H.WM.vs.MG.Go.withMetabolic.top10.barplot_220701.png",width = 8,height = 8,dpi = 300)

In [7]:
library(ggpubr)
library(ggrepel)
test$celltype<-ifelse(test$log_fc_est>0,"Astrocyte.Gfap","Astrocyte.Slc7a10")
### reverse the direction 
test$log_fc_est<--test$log_fc_est

In [ ]:
### vocalno plot
pdf("13cp.sham_H.Astr.DEG.vocalno.withlabel_reversed220701.plot.pdf",width = 15,height = 15)
options(repr.plot.width=15,repr.plot.height=15)
ggplot(test,aes(x=log_fc_est,y=-log10(p),col=celltype,label=rownames(test)
               ))+
    geom_point(size=2)+
    geom_text_repel(max.overlaps = 18)+
    scale_color_manual(values = c("#911eb4","#DE8BE6"))+
    theme_minimal()+
    theme(panel.grid =  element_blank())+
    geom_vline(xintercept = c(-0.5,0.5),col="grey")+
    geom_hline(yintercept = -log10(0.05),col="grey")
dev.off()
#ggsave("13cp.sham_H.Astr.DEG.vocalno.plot.png",width = 15,height = 9,dpi = 400)

#### related regulons

In [ ]:
gfap_deg<-rownames(test[test$celltype=="Astrocyte.Gfap",])
gfap_deg
slc_deg<-rownames(test[test$celltype=="Astrocyte.Slc7a10",])
slc_deg

mods<-list(gfap=gfap_deg,slc=slc_deg)
mods

In [ ]:
library(KernSmooth)
library(RColorBrewer)
library(plotly)
library(BiocParallel)
library(grid)
#library(ComplexHeatmap)
library(data.table)
library(dplyr)
library(tidyverse)
library(pheatmap)
library(scales)

In [260]:
regulon_list<-readRDS('/home/jovyan/zxli_SCI/result/WGCNA/WT.SCT.res02.domain_220311/topregulon/515.regulon.genes.list.rds')

In [ ]:
r_genes<-unique(unlist(regulon_list))
length(r_genes)

In [ ]:
backgenes<-readRDS("/home/jovyan/zxli_SCI/result/cellchat/time_domain_220325/WT.SCT.data.rds")
backgenes<-rownames(backgenes)
head(backgenes)

In [ ]:
total_genes<-backgenes#unique(unlist(regulon_list))
t_l<-length(total_genes)
t_l

In [268]:
### create hypergeometric dataframe for each GOI(gene of interest)
mod_regulon_df_list<-list()
#inter_mod_reg_df<-data.frame(row.names = names(mods),module=names(mods))
#c<-2
f<-"astr_deg.regulon.intersect_genes/"
if(!dir.exists(f))
    dir.create(f)

for(i in names(regulon_list)){
    inter_list<-list()
    #inter_mod_reg_df[,i]<-NA
    r_genes<-regulon_list[[i]]
    #r_genes<-intersect(r_genes,intgenes) ### filter regulon genes that not exist in all module gene background
    k<-length(r_genes)
    # find the module success gene number in target regulon,regulon number, remaining regulons size and module size
    hyper_df <- data.frame(matrix(nrow=length(mods),ncol = 9),row.names = names(mods))
    

    colnames(hyper_df) <- c("q",  #regulon in module
                            "m",  #module size
                            "n",  #regulon in background 
                            "k",  #regulon size
                            "p.val",
                            "p.val.ad",
                            "sig",
                            "ratio",
                            "score")

    for(j in names(mods)){
        m_genes<-mods[[j]]
        suc_genes<-intersect(m_genes,r_genes)
        suc_genes_l<-length(suc_genes)
        m<-length(m_genes)
        n<-t_l-m
        p<-round(phyper(suc_genes_l,m,n,k,lower.tail = F,log.p = F),4)
        r<-round(suc_genes_l/k,4)
        
        if(suc_genes_l==0){
            inter_list[[j]]<-""
        }else{
            inter_list[[j]]<-c(suc_genes)
        }
        
        
        hyper_df[j,c("q","m","n","k","p.val")]<-c(suc_genes_l,m,n,k,p)
        hyper_df$ratio<-round(hyper_df$q/hyper_df$k,4) #calculate the regulon enriched ratio in each module
        #hyper_df<-subset(hyper_df,ratio>0.2) #subset regulons that have 20% enrichment ratio in target modules
    }
    
    saveRDS(inter_list,paste0(f,i,".intersect_genes.in.astr_deg.rds"))
    #inter_mod_reg_df<-as.data.frame(cbind(inter_mod_reg_df,inter_list))
    #colnames(inter_mod_reg_df)[c]<-i
    
    hyper_df$p.val.ad<-round(p.adjust(hyper_df$p.val,"bonferroni"),4)
    hyper_df$sig<-round((1-hyper_df$p.val.ad)/0.05,4)
    hyper_df[hyper_df$p.val.ad>=0.01,"sig"]<-0
    hyper_df$score<-hyper_df$sig*hyper_df$ratio
    mod_regulon_df_list[[i]]<-hyper_df
    #mod_regulon_df_list[[i]]$p.val.ad<-
    #mod_regulon_df_list[[i]]$sig<-round(mod_regulon_df_list[[i]])
    #dim(df)
    #head(df)
    #c<-c+1
}


In [ ]:
### create module-regulon p.val matrix
p_ma<-do.call(cbind.data.frame,mod_regulon_df_list)
head(p_ma)
p_ma<-as.matrix(p_ma[,grepl(".sig",colnames(p_ma))])
colnames(p_ma)<-gsub(".sig","",colnames(p_ma))
p_ma<-t(p_ma)
head(p_ma)

In [ ]:
# create ratio matrix
r_ma<-do.call(cbind.data.frame,mod_regulon_df_list)
head(r_ma)
r_ma<-as.matrix(r_ma[,grepl("ratio",colnames(r_ma))])
colnames(r_ma)<-gsub(".ratio","",colnames(r_ma))
r_ma<-t(r_ma)
head(r_ma)

In [ ]:
s_ma<-do.call(cbind.data.frame,mod_regulon_df_list)
head(s_ma)
s_ma<-as.matrix(s_ma[,grepl("score",colnames(s_ma))])
colnames(s_ma)<-gsub(".score","",colnames(s_ma))
s_ma<-t(s_ma)
head(s_ma)

In [274]:
write.csv(p_ma,"astrDEG_regulon.hyper.pad001.pval.ad_significance.matrix.csv")
write.csv(s_ma,"astrDEG-regulon.hyper.pad001.pval-ratio.score.matrix.csv")

In [275]:
saveRDS(mod_regulon_df_list,"astrDEG.regulon.hypergeomic.test.list.pad001.rds")

In [24]:
mod_regulon_df_list<-readRDS("astrDEG.regulon.hypergeomic.test.list.pad001.rds")

In [ ]:
df<-do.call(rbind.data.frame,mod_regulon_df_list)
head(df)
df$module<-str_split(rownames(df),"\\.",simplify = T)[,2]
df$regulon<-str_split(rownames(df),"\\.",simplify = T)[,1]
##df<-df[!df$module=="",]
head(df)
dim(df)

In [ ]:
df_sub<-df[df$p.val.ad<0.01&df$q>=5&df$ratio>0.01,]
dim(df_sub)

In [27]:
top5<-df_sub %>% group_by(.,module) %>% top_n(15,wt = -p.val.ad) %>% top_n(15,wt = ratio)

In [28]:
top5<- top5 %>% group_by(.,module) %>% arrange(.,module)
top5$regulon<-factor(top5$regulon,levels = unique(top5$regulon))

In [ ]:
options(repr.plot.width=4,repr.plot.height=5)
ggplot(top5,aes(x=regulon,y=module))+
    geom_point(aes(size=q,color=sig#,shape=sc_shared
                  ))+
    scale_color_gradientn(colours = c("darkblue","pink","red"))+
    theme_bw()+#ggtitle(label = "72h")+
    theme(axis.text.x = element_text(angle = 60,size = 12,vjust = 1,hjust = 1),
          axis.text.y.left = element_text(angle = 30,size = 12),
          axis.title = element_blank(),
         #legend.position="bottom",
         # legend.back=element_rect(size = 0.005)
         )#+
        #facet_wrap(~time,nrow = 1,ncol = 4)

#ggsave("astrDEG.regulon.pad001.num5.pad_ratio_top15.pointplot.png",width = 4,height = 2.6,dpi = 300)